In [57]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from src.data.data_fetcher import get_all_features, get_raw_data
from src.features.feature_engineering import prepare_data

%pip install tensorflow
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Bruker\OneDrive\NTNU semester 05\TDT4173 Maskinlæring\ml_power_predictor\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [58]:
train_a, train_b, train_c, X_train_estimated_a, X_train_estimated_b, X_train_estimated_c, X_train_observed_a, X_train_observed_b, X_train_observed_c, X_test_estimated_a, X_test_estimated_b, X_test_estimated_c = get_raw_data()

In [59]:
targets = train_a
features_observed = X_train_observed_a
features_estimated = X_train_estimated_a
features_test = X_test_estimated_a
targets.head(), features_observed.head(), features_estimated.head(), features_test.head()

(                 time  pv_measurement
 0 2019-06-02 22:00:00            0.00
 1 2019-06-02 23:00:00            0.00
 2 2019-06-03 00:00:00            0.00
 3 2019-06-03 01:00:00            0.00
 4 2019-06-03 02:00:00           19.36,
         date_forecast  absolute_humidity_2m:gm3  air_density_2m:kgm3  \
 0 2019-06-02 22:00:00                       7.7                1.230   
 1 2019-06-02 22:15:00                       7.7                1.229   
 2 2019-06-02 22:30:00                       7.7                1.228   
 3 2019-06-02 22:45:00                       7.7                1.226   
 4 2019-06-02 23:00:00                       7.7                1.225   
 
    ceiling_height_agl:m  clear_sky_energy_1h:J  clear_sky_rad:W  \
 0           1744.900024                    0.0              0.0   
 1           1734.000000                    0.0              0.0   
 2           1723.500000                    0.0              0.0   
 3           1713.400024                    0.0      

In [60]:

# Call the function with the file paths
X_train, y_train, X_val, y_val, X_test, train_data, val_data = prepare_data(targets, 
                                                                            features_observed, 
                                                                            features_estimated, 
                                                                            features_test)

# Check the prepared DataFrames
X_train.head(1), y_train.head(1), X_val.head(1), y_val.head(1), X_test.head(1), train_data.head(1), val_data.head(1)


(                     absolute_humidity_2m:gm3  air_density_2m:kgm3  \
 2019-12-01 09:00:00                       4.8                1.272   
 
                      ceiling_height_agl:m  clear_sky_energy_1h:J  \
 2019-12-01 09:00:00                456.25                27331.5   
 
                      clear_sky_rad:W  cloud_base_agl:m  dew_or_rime:idx  \
 2019-12-01 09:00:00        17.549999            456.25              0.0   
 
                      dew_point_2m:K  diffuse_rad:W  diffuse_rad_1h:J  ...  \
 2019-12-01 09:00:00      273.150024          5.175       9751.150391  ...   
 
                      sun_azimuth:d  sun_elevation:d  \
 2019-12-01 09:00:00     155.780746          2.32725   
 
                      super_cooled_liquid_water:kgm2  t_1000hPa:K  \
 2019-12-01 09:00:00                            0.25   274.350006   
 
                      total_cloud_cover:p  visibility:m  wind_speed_10m:ms  \
 2019-12-01 09:00:00                100.0   7155.975098               4.

In [61]:
# Drop the date column
X_val = X_val.drop(['date_calc'], axis=1)

# Scale the feature data
scaler_X = MinMaxScaler(feature_range=(0, 1))
scaler_Y = MinMaxScaler(feature_range=(0, 1))

X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_Y.fit_transform(y_train.values.reshape(-1, 1))
X_val_scaled = scaler_X.transform(X_val)
y_val_scaled = scaler_Y.transform(y_val.values.reshape(-1, 1))

# Reshape input to be [samples, time steps, features]
X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], 1, X_train_scaled.shape[1]))
X_val_reshaped = X_val_scaled.reshape((X_val_scaled.shape[0], 1, X_val_scaled.shape[1]))

# Construct the LSTM model
model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(50, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mean_absolute_error')

# Train the model
history = model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, validation_data=(X_val_reshaped, y_val_scaled), verbose=2, shuffle=False)

# Make predictions on validation data
y_val_pred_scaled = model.predict(X_val_reshaped)

# Transform predictions back to original scale
y_val_pred = scaler_Y.inverse_transform(y_val_pred_scaled).flatten()

# Calculate Mean Absolute Error on validation data
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_val

Epoch 1/50
17/17 - 4s - loss: 0.0793 - val_loss: 0.0693 - 4s/epoch - 228ms/step
Epoch 2/50
17/17 - 0s - loss: 0.0670 - val_loss: 0.0558 - 145ms/epoch - 9ms/step
Epoch 3/50
17/17 - 0s - loss: 0.0488 - val_loss: 0.0496 - 150ms/epoch - 9ms/step
Epoch 4/50
17/17 - 0s - loss: 0.0409 - val_loss: 0.0421 - 157ms/epoch - 9ms/step
Epoch 5/50
17/17 - 0s - loss: 0.0372 - val_loss: 0.0380 - 121ms/epoch - 7ms/step
Epoch 6/50
17/17 - 0s - loss: 0.0331 - val_loss: 0.0421 - 132ms/epoch - 8ms/step
Epoch 7/50
17/17 - 0s - loss: 0.0499 - val_loss: 0.0361 - 168ms/epoch - 10ms/step
Epoch 8/50
17/17 - 0s - loss: 0.0588 - val_loss: 0.0467 - 302ms/epoch - 18ms/step
Epoch 9/50
17/17 - 0s - loss: 0.0381 - val_loss: 0.0386 - 143ms/epoch - 8ms/step
Epoch 10/50
17/17 - 0s - loss: 0.0437 - val_loss: 0.0437 - 124ms/epoch - 7ms/step
Epoch 11/50
17/17 - 0s - loss: 0.0395 - val_loss: 0.0482 - 103ms/epoch - 6ms/step
Epoch 12/50
17/17 - 0s - loss: 0.0549 - val_loss: 0.0422 - 159ms/epoch - 9ms/step
Epoch 13/50
17/17 - 0s -

86.59466305505845